## Simulate running on multiple GPUs

In [1]:
from dask.distributed import Client

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('/home/ubuntu/adadamp-experiments/')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
import importlib
import pandas as pd
from time import time
from copy import copy

In [4]:
# import simulator
os.chdir('/home/ubuntu/adadamp-experiments/')
classifier = importlib.import_module("exp-dask.classifier")
from classifier import DaskClassifierSimulator

In [5]:
# training client
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/2912/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform_test)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [8]:
def write_stats(hist, directory, filename):
    toCSV = hist
    with open('./exp-dask/{}/{}'.format(directory, filename), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
        fc.writeheader()
        fc.writerows(toCSV)

In [9]:
def train(model, train_set, test_set, stats_df, n_epochs=200, log_interval=1, stats_dir=''):
    """
    Train based on expiriment params
    
    Parameters:
    epoch_sched: update lr and bs at epochs in this list
    lr_sched: update lr to value at matching epoch. Should be same length as epoch_sched
    bs_sched: update bs to value at matching epoch. Should be same length as epoch_sched
    """
    
    stats = list(stats_df.T.to_dict().values())
    print("[SETUP] Loaded {} epochs of stats".format(len(stats)))
    
    # preprocess data
    start = time()
    train_set = model.preprocess(train_set)
    test_set = model.preprocess(test_set)
    print("[SETUP] Pre-Processed in {} seconds".format(time() - start))
    
    history = []
    for epoch in range(min(n_epochs, len(stats))):
        
        start = time()
        
        # set stats
        stat = stats.pop(0)
        model.set_sim(stat)
        
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {} in {} seconds".format(score, time() - start))
        history.append(datum)
        
        if epoch % log_interval == 0:
            write_stats(history, stats_dir, 'results-ep{}.csv'.format(epoch))

    return history

In [10]:
def run_sim(stats_df_loc, out_folder, timings, max_bs=99999999):
    
    print("Running simulation....\n\n")
    start = time()
    
    stats_df = pd.read_csv(stats_df_loc)
    
    if 'lr_' in stats_df.columns:
        stats_df = stats_df.rename(columns={"lr_": "partial_fit__lr", "damping_": "partial_fit__batch_size"})
    
    init_lr = stats_df['partial_fit__lr'][0]
    init_bs = stats_df['partial_fit__batch_size'][0]
    momentum = stats_df['optimizer__momentum'][0]
    device = "cpu" if not torch.cuda.is_available() else "cuda:0"

    args = dict(
        module=Wide_ResNet,
        module__depth=16,
        module__widen_factor=4,
        module__dropout_rate=0.3,
        module__num_classes=len(classes),
        loss=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.SGD,
        optimizer__lr=init_lr,
        optimizer__momentum=momentum,
        optimizer__nesterov=True,
        optimizer__weight_decay=0.5e-3,
        batch_size=init_bs,
        max_epochs=200,
        device=device,
        grads_per_worker=128,
        client=client,
        lr=init_lr,
        max_batch_size=max_bs
    )
    model = DaskClassifierSimulator(**args)
    model.set_times(timings['mult'], timings['score'], timings['deepcopy'], timings['grad128'])
    
    # train
    hist = train(model, 
             train_set, 
             test_set, 
             stats_df,
             n_epochs=200, 
             log_interval=20,
             stats_dir=out_folder
            )
    write_stats(hist, out_folder, 'results-final.csv')
    
    print('Finished simulation in {} seconds'.format(time() - start))
    

In [12]:
# test exp
timings = { 'mult': True, 'score': 0.0, 'deepcopy': 0.0, 'grad128': 0.0 }
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/test'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.534586906433105 seconds
[Epoch 0] Score: 0.5943999886512756 in 10.096720933914185 seconds
[Epoch 1] Score: 0.6757999658584595 in 7.93016505241394 seconds
[Epoch 2] Score: 0.7128999829292297 in 6.990623235702515 seconds
Finished simulation in 43.66578984260559 seconds


In [11]:
## TIMINGS FOR NORMAL EXPERIMENTS
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.05855  # seconds
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'mult': False, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [18]:
# exp 1 - dec-lr-1-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 17.500604391098022 seconds
[Epoch 0] Score: 0.5990999937057495 in 60.29955196380615 seconds
[Epoch 1] Score: 0.6762999892234802 in 60.30539417266846 seconds
[Epoch 2] Score: 0.7156999707221985 in 59.774914503097534 seconds
[Epoch 3] Score: 0.7777999639511108 in 60.207035779953 seconds
[Epoch 4] Score: 0.7678999900817871 in 59.78869414329529 seconds
[Epoch 5] Score: 0.7994999885559082 in 60.308109283447266 seconds
[Epoch 6] Score: 0.8030999898910522 in 59.963244676589966 seconds
[Epoch 7] Score: 0.8100000023841858 in 60.10037112236023 seconds
[Epoch 8] Score: 0.8186999559402466 in 59.96827411651611 seconds
[Epoch 9] Score: 0.8077999949455261 in 60.28385829925537 seconds
[Epoch 10] Score: 0.8211999535560608 in 60.00210118293762 seconds
[Epoch 11] Score: 0.8321999907493591 in 60.082077980041504 seconds
[Epoch 12] Score: 0.8306999802589417 in 59.640068769454956 seconds
[Epoch 13] Score: 0.8325999975204468 

In [25]:
# exp 5 - large bs 1 1 machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-1-machine'
run_sim(stats_path, out_path, timings, max_bs=5120)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 17.082221746444702 seconds
[Epoch 0] Score: 0.5884999632835388 in 30.94543957710266 seconds
[Epoch 1] Score: 0.6516000032424927 in 30.91340970993042 seconds
[Epoch 2] Score: 0.7285000085830688 in 30.86415672302246 seconds
[Epoch 3] Score: 0.7662000060081482 in 31.18868088722229 seconds
[Epoch 4] Score: 0.7896999716758728 in 30.9765408039093 seconds
[Epoch 5] Score: 0.7972999811172485 in 30.914204597473145 seconds
[Epoch 6] Score: 0.8024999499320984 in 31.085127592086792 seconds
[Epoch 7] Score: 0.8089999556541443 in 30.978079080581665 seconds
[Epoch 8] Score: 0.8148999810218811 in 30.929124355316162 seconds
[Epoch 9] Score: 0.8120999932289124 in 30.941798210144043 seconds
[Epoch 10] Score: 0.8294000029563904 in 30.995038747787476 seconds
[Epoch 11] Score: 0.8185999989509583 in 30.99557065963745 seconds
[Epoch 12] Score: 0.8331999778747559 in 30.947920083999634 seconds
[Epoch 13] Score: 0.84169995784759

In [ ]:
# exp 2 - inc-bs
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.031914472579956 seconds
[Epoch 0] Score: 0.5943999886512756 in 67.52644038200378 seconds
[Epoch 1] Score: 0.6757999658584595 in 59.98326015472412 seconds
[Epoch 2] Score: 0.7128999829292297 in 59.854674100875854 seconds
[Epoch 3] Score: 0.7423999905586243 in 59.890366554260254 seconds
[Epoch 4] Score: 0.7707999944686891 in 60.007487058639526 seconds
[Epoch 5] Score: 0.7875999808311462 in 60.24218225479126 seconds
[Epoch 6] Score: 0.7892999649047852 in 59.979164361953735 seconds
[Epoch 7] Score: 0.8071999549865723 in 60.02729058265686 seconds
[Epoch 8] Score: 0.8120999932289124 in 59.98218131065369 seconds
[Epoch 9]

In [ ]:
# exp 3 - hybrid 1 1 machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-1-machine'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 4 - hybrid 2 1 machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-1-machine'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 6 - large bs 2 1 machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-1-machine'
run_sim(stats_path, out_path, timings, max_bs=5120)

In [ ]:
# timings for multi machine set ups
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.05855  # seconds
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'mult': True, 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [ ]:
# exp 7 - dec-lr-P-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 8 - increasing batch p machines
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 9 - hybrid 1 P machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 10 - hybrid 2 P machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-mult-machines'
run_sim(stats_path, out_path, timings)

In [ ]:
# exp 11 - large bs 1 P machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-mult-machines'
run_sim(stats_path, out_path, timings, max_bs=5120)

In [ ]:
# exp 12 - large bs 2 P machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-mult-machines'
run_sim(stats_path, out_path, timings, max_bs=5120)